In [1]:
import json
import os
import pandas as pd 
import numpy as np
from datetime import date, datetime
import pytz

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
file_date = date.today().strftime("%Y%m%d")
file_date

'20200417'

In [4]:
pd.set_option('display.max_rows', 500)

In [5]:
with open("alert.json", "r") as read_file:
    data = json.load(read_file)

In [6]:
data['header']['timestamp']
time = pd.to_datetime(data['header']['timestamp'], unit='s')
time = time + pd.Timedelta('10:00:00')
datetime = time.strftime('%Y%m%d %H:%M:%S')

In [7]:
entities = data['entity']
line_service_alerts = []
station_facilities_alerts = []
trip_based_alerts = []

In [8]:
for line_service_alert in entities:
    alerts = line_service_alert['alert']['informedEntity']
    keep_running = True
    for alert in alerts:
        if 'routeId' in alert and keep_running == True:
            line_service_alerts.append(line_service_alert)
            keep_running = False 
        else:
            break  
len(line_service_alerts)

1

In [9]:
for station_facilities_alert in entities:
    alerts = station_facilities_alert['alert']['informedEntity']
    keep_running = True
    for alert in alerts:
        if 'stopId' in alert and keep_running == True:
            station_facilities_alerts.append(station_facilities_alert)
            keep_running = False
            
len(station_facilities_alerts)

5

In [10]:
for trip_based_alert in entities:
    alerts = trip_based_alert['alert']['informedEntity']
    keep_running = True
    for alert in alerts:
        if 'trip' in alert and keep_running == True:
            trip_based_alerts.append(trip_based_alert)
            keep_running = False
            
len(trip_based_alerts)

2

In [11]:
trip_based_alerts[0]

{'id': '3',
 'alert': {'informedEntity': [{'agencyId': 'SydneyTrains',
    'trip': {'tripId': '32-H.1648.102.96.K.8.61930299'}},
   {'agencyId': 'SydneyTrains',
    'trip': {'tripId': '13-F.1648.102.64.B.8.62240900'}},
   {'agencyId': 'SydneyTrains',
    'trip': {'tripId': '3--E.1648.102.64.B.8.61930303'}},
   {'agencyId': 'SydneyTrains',
    'trip': {'tripId': '8--H.1648.102.112.B.8.61257377'}}],
  'url': {'translation': [{'text': 'https://transportnsw.info/alerts#/train',
     'language': 'en'}]},
  'headerText': {'translation': [{'text': 'Delayed', 'language': 'en'}]},
  'descriptionText': {'translation': [{'text': ' due to an operational issue at Central earlier',
     'language': 'en'}]}}}

In [12]:
station_facilities_alerts[0]

{'id': '9',
 'alert': {'informedEntity': [{'agencyId': 'SydneyTrains',
    'stopId': '223010'}],
  'cause': 'OTHER_CAUSE',
  'effect': 'UNKNOWN_EFFECT',
  'url': {'translation': [{'text': 'https://transportnsw.info/alerts#/train',
     'language': 'en'}]},
  'headerText': {'translation': [{'text': 'Lift Availability - Woolooware',
     'language': 'en'}]},
  'descriptionText': {'translation': [{'text': 'The lift between the concourse and Denham Avenue is temporarily out of service. If you require assistance, please ask staff or phone 9522 1265.',
     'language': 'en'}]}}}

In [13]:
line_service_alerts[0]

{'id': '1',
 'alert': {'informedEntity': [{'agencyId': 'NSWTrains', 'routeId': 'BMT_2'},
   {'agencyId': 'NSWTrains', 'routeId': 'BMT_1'}],
  'url': {'translation': [{'text': 'https://transportnsw.info/alerts#/train',
     'language': 'en'}]},
  'headerText': {'translation': [{'text': 'Trackwork - Blue Mountains Line',
     'language': 'en'}]},
  'descriptionText': {'translation': [{'text': 'Until further notice:\n- Due to infrastructure damage caused by bushfires, buses continue to replace trains between Lithgow and Mount Victoria.\n- Trains continue to run between Bathurst, Lithgow, and and Central, to a changed timetable.\n\nPlease use transport apps or visit transportnsw.info for details.',
     'language': 'en'}]}}}

In [14]:
station_facilities_alerts_agencyId = []
station_facilities_alerts_stopId = []
station_facilities_alerts_cause = []
station_facilities_alerts_effect = []
station_facilities_alerts_headerText = []
station_facilities_alerts_descriptionText = []
station_facilities_alerts_datetime = []
for entity in station_facilities_alerts:
        station_facilities_alerts_datetime.append(datetime)
        station_facilities_alerts_agencyId.append(entity['alert']['informedEntity'][0]['agencyId'])
        station_facilities_alerts_stopId.append(entity['alert']['informedEntity'][0]['stopId'])
        station_facilities_alerts_cause.append(entity['alert']['cause'])
        station_facilities_alerts_effect.append(entity['alert']['effect'])
        station_facilities_alerts_headerText.append(entity['alert']['headerText']['translation'][0]['text'])
        station_facilities_alerts_descriptionText.append(entity['alert']['descriptionText']['translation'][0]['text'])

In [15]:
df_station = pd.DataFrame(columns = ['datetime','agencyId',
                                     'stopId',
                                     'cause',
                                     'effect',
                                     'headerText',
                                     'descriptionText'])

df_station['datetime'] = pd.Series(station_facilities_alerts_datetime)
df_station['agencyId'] = pd.Series(station_facilities_alerts_agencyId)
df_station['stopId'] = pd.Series(station_facilities_alerts_stopId)
df_station['cause'] = pd.Series(station_facilities_alerts_cause)
df_station['effect'] = pd.Series(station_facilities_alerts_effect)
df_station['headerText'] = pd.Series(station_facilities_alerts_headerText)
df_station['descriptionText'] = pd.Series(station_facilities_alerts_descriptionText)

In [16]:
df_station.head()

,datetime,agencyId,stopId,cause,effect,headerText,descriptionText
0,20200417 08:51:52,SydneyTrains,223010,OTHER_CAUSE,UNKNOWN_EFFECT,Lift Availability - Woolooware,The lift between the concourse and Denham Aven...
1,20200417 08:51:52,SydneyTrains,257610,OTHER_CAUSE,UNKNOWN_EFFECT,Station Update - Bowral,Drop-off zones on Kirkham Road alongside platf...
2,20200417 08:51:52,SydneyTrains,200050,OTHER_CAUSE,UNKNOWN_EFFECT,Station Update - St James,The public toilets at St James Station are clo...
3,20200417 08:51:52,SydneyTrains,204420,OTHER_CAUSE,UNKNOWN_EFFECT,Station Update - Sydenham,New Metro Platforms are being constructed on p...
4,20200417 08:51:52,SydneyTrains,200080,OTHER_CAUSE,UNKNOWN_EFFECT,Station Update - Wynyard,"Station Upgrade - Customers, due to Wynyard st..."


In [17]:
hdr = False if os.path.isfile('station_facilities_alerts_' + file_date + '.csv') else True
df_station.to_csv('station_facilities_alerts_' + file_date + '.csv', mode='a', header=hdr, index=False)



In [18]:
line_service_alerts_agencyId = []
line_service_alerts_routeId = []
line_service_alerts_headerText = []
line_service_alerts_descriptionText = []
line_service_alerts_datetime = []
for entity in line_service_alerts:
    for sub_entity in entity['alert']['informedEntity']:
        line_service_alerts_datetime.append(datetime)
        line_service_alerts_agencyId.append(sub_entity['agencyId'])      
        line_service_alerts_routeId.append(sub_entity['routeId'])
        line_service_alerts_headerText.append(entity['alert']['headerText']['translation'][0]['text'])
        line_service_alerts_descriptionText.append(entity['alert']['descriptionText']['translation'][0]['text'])     

In [19]:
df_line = pd.DataFrame(columns = ['datetime','agencyId',
                                  'routeId',
                                  'headerText',
                                  'descriptionText'])

df_line['datetime'] = pd.Series(line_service_alerts_datetime)
df_line['agencyId'] = pd.Series(line_service_alerts_agencyId)
df_line['routeId'] = pd.Series(line_service_alerts_routeId)
df_line['headerText'] = pd.Series(line_service_alerts_headerText)
df_line['descriptionText'] = pd.Series(line_service_alerts_descriptionText)

In [20]:
df_line.head()

,datetime,agencyId,routeId,headerText,descriptionText
0,20200417 08:51:52,NSWTrains,BMT_2,Trackwork - Blue Mountains Line,Until further notice:\n- Due to infrastructure...
1,20200417 08:51:52,NSWTrains,BMT_1,Trackwork - Blue Mountains Line,Until further notice:\n- Due to infrastructure...


In [21]:
hdr = False if os.path.isfile('line_service_alerts_' + file_date + '.csv') else True
df_line.to_csv('line_service_alerts_' + file_date + '.csv', mode='a', header=hdr, index=False)



In [22]:
trip_based_alerts[0]

{'id': '3',
 'alert': {'informedEntity': [{'agencyId': 'SydneyTrains',
    'trip': {'tripId': '32-H.1648.102.96.K.8.61930299'}},
   {'agencyId': 'SydneyTrains',
    'trip': {'tripId': '13-F.1648.102.64.B.8.62240900'}},
   {'agencyId': 'SydneyTrains',
    'trip': {'tripId': '3--E.1648.102.64.B.8.61930303'}},
   {'agencyId': 'SydneyTrains',
    'trip': {'tripId': '8--H.1648.102.112.B.8.61257377'}}],
  'url': {'translation': [{'text': 'https://transportnsw.info/alerts#/train',
     'language': 'en'}]},
  'headerText': {'translation': [{'text': 'Delayed', 'language': 'en'}]},
  'descriptionText': {'translation': [{'text': ' due to an operational issue at Central earlier',
     'language': 'en'}]}}}

In [23]:
trip_based_alerts_agencyId = []
trip_based_alerts_tripId = []
trip_based_alerts_headerText = []
trip_based_alerts_descriptionText = []
trip_based_alerts_datetime = []
for entity in trip_based_alerts:
    for sub_entity in entity['alert']['informedEntity']:
        trip_based_alerts_datetime.append(datetime)
        trip_based_alerts_agencyId.append(sub_entity['agencyId'])
        trip_based_alerts_tripId.append(sub_entity['trip']['tripId'])
        trip_based_alerts_headerText.append(entity['alert']['headerText']['translation'][0]['text'])
        trip_based_alerts_descriptionText.append(entity['alert']['descriptionText']['translation'][0]['text'])

In [24]:
df_trip = pd.DataFrame(columns = ['datetime','agencyId',
                                  'tripId',
                                  'headerText',
                                  'descriptionText'])
df_trip['agencyId'] = pd.Series(trip_based_alerts_agencyId)
df_trip['datetime'] = pd.Series(trip_based_alerts_datetime)
df_trip['tripId'] = pd.Series(trip_based_alerts_tripId)
df_trip['headerText'] = pd.Series(trip_based_alerts_headerText)
df_trip['descriptionText'] = pd.Series(trip_based_alerts_descriptionText)

In [25]:
df_trip.head()

,datetime,agencyId,tripId,headerText,descriptionText
0,20200417 08:51:52,SydneyTrains,32-H.1648.102.96.K.8.61930299,Delayed,due to an operational issue at Central earlier
1,20200417 08:51:52,SydneyTrains,13-F.1648.102.64.B.8.62240900,Delayed,due to an operational issue at Central earlier
2,20200417 08:51:52,SydneyTrains,3--E.1648.102.64.B.8.61930303,Delayed,due to an operational issue at Central earlier
3,20200417 08:51:52,SydneyTrains,8--H.1648.102.112.B.8.61257377,Delayed,due to an operational issue at Central earlier
4,20200417 08:51:52,SydneyTrains,33-D.1648.102.112.K.8.61261475,Cancelled,due to a train requiring mechanical repairs e...


In [26]:
hdr = False if os.path.isfile('trip_based_alerts_' + file_date + '.csv') else True
df_trip.to_csv('trip_based_alerts_' + file_date + '.csv', mode='a', header=hdr, index=False)
